In [4]:
# request 唯一的作用是，模拟浏览器发送请求

## 爬取sogou首页的数据

In [6]:
# 指定URL
url = 'https://www.sogou.com/'

# 发请求
response = requests.get(url = url)

# 获取响应数据, 只有请求成功之后才能返回这个响应数据
page_text = response.text # 返回一个字符串

# 持久化存储
with open('./sogou.html', 'w', encoding = 'utf-8') as fp:
          fp.write(page_text)

print('数据爬取结束')

数据爬取结束


In [ ]:
# 但是这个网页丧失了样式，但是数据都还在

##  设计一个简单的网页采集器，搜狗爬取关键词结果页面

In [11]:
# 网页是'https://www.sogou.com/web?query=李振宁&_ast=1619735760&_asf=www.sogou.com&w=01029901&p=40040100&dp=1&cid=&s_from=result_up&sut=3110&sst0=1619735766394&lkt=0%2C0%2C0&sugsuv=00D219E62EC142EE6070667BE4E8E543&sugtime=1619735766394'
# 但是这个是静态的网页，而且如果只写到web?query=李振宁的话照样请求不了，所以需要一些处理
# 处理url携带的参数，多组参数

# 这里涉及到一个UA伪装，就是user_agents伪装，因为用requests来爬的话就是用requests的身份来爬，而加上UA后就是用你电脑的身份爬了
# UA伪装：门户网站会检测对应请求的身份标识，当门户网站发现是某一个浏览器，就说明该请求是正常的，一定不会拒绝
# 如果监测到请求不是基于浏览器，则表示该请求为不正常。
# 我目前的身份是：Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36

# UA伪装，将对应的user_agents封装到一个字典中（算头信息，所以用header）这里的header基于谷歌浏览器。
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

url = 'https://www.sogou.com/web'

# 将url携带的参数，封装到字典中，这样以后就可以直接输入然后去搜索了
kw = input('enter a word:')
params = {
    'query':kw
}

# 目前发起请求的是携带参数的，并且已经处理了(动态拼接)
# 进行UA伪装
requests.get(url = url, params = params, headers = headers)

page_text = response.text

fileName = '搜狗-' + kw + '.md'
with open(fileName, 'w', encoding = 'utf-8') as fp:
          fp.write(page_text)

print(fileName, '保存成功')

enter a word:lizh
搜狗-lizh.md 保存成功


##  破解百度翻译

In [44]:
# 使用百度翻译的时候，发现会局部刷新，url改变。所以需要用f12看一下ajax的请求。
# 每一个ajax对应的是每一个字符的翻译
# 需要按F12，然后找network的all右边的XHR，这个指ajax返回的信息，然后找到你请求的项目（逐个点击然后往下拉，看到自己最终想要翻译的句子）

# 对应的请求是POST请求，所以需要用requests做POST请求。
# 响应数据是json数据，是需要捕获的数据。
import requests

post_url = 'https://fanyi.baidu.com/sug' # 从f12的request url中获取

# UA伪装
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

# 这个data在f12最下方的Form Data中看到，也就是我们想翻译的浏览器上传的数据！
translate = input('input some words you want to translate to Chinese: ')
data = {
    'kw': translate
}

response = requests.post(url = post_url, data = data, headers = headers)


# 百度翻译响应回来的数据是一组json，因为从F12的Response Header中的Content-Type: application/json看出来！
# 用.json()直接返回的是obj（一定要之前确定服务器响应的是json类型的才行！）
dict_obj = response.json() # 这个返回的是翻译结果

# 存储
# fileName = './' + translate + '.json'
# fp = open(fileName, 'w', encoding = 'utf-8')

# json.dump()就是指对json文件进行编码！
# json.dump(dict_obj, fp = fp, ensure_ascii = False) # 因为有中文的地方不能用ASCII码！

# fp.close() # 这句话一定要加，不然显示大小为0，永远在打开的状态！


# 这里并不想存储，直接返回翻译结果：
print('\nThe meaning is: ')

# 防止返回的是空值
if dict_obj['data'] != []:
    print(dict_obj['data'][0]['v'])
else:
    print('\'',translate, '\'', 'Can\'t be translated!')

print('\n数据查询结束')



input some words you want to translate to Chinese: boring

The meaning is: 
adj. 无聊的，无趣的; 令人厌烦的; 单调的，乏味的 n. 钻孔; 钻屑 v. 令人厌烦(bor

数据查询结束


##  豆瓣电影分类排行榜

In [69]:
import requests

# 爬取豆瓣喜剧电影的排行榜的名字
# 打开网页，复制f12中的url
url = 'https://movie.douban.com/j/chart/top_list'

# 这个参数是从f12中的最下面的那个地方Query String Parameters找到的
param = {
    'type': '24', # 这里注意，所有的东西都要用string的形式！
    'interval_id': '100:90',
    'action': '', 
    'start': '0', # 这个大概意思就是从第0个开始取，再取10个
    'limit': '10'
}

# UA伪装
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

response = requests.get(url = url, params = param, headers = headers) # 这里用get是因为f12中显示的是get类型
list_data = response.json()

# 此时所有页面的信息都已经有了，现在把电影名列出来
movie_name = []
movie_rating = []
for i in range(len(list_data)):
    movie_name.append(list_data[i]['title'])
    movie_rating.append(float(list_data[i]['rating'][0]))

movie_info_dict = dict(zip(movie_name, movie_rating)) # 生成一个字典
movie_info_dict

{'憨豆先生精选辑': 9.6,
 '美丽人生': 9.5,
 '福尔摩斯二世': 9.5,
 '黄子华栋笃笑之金盆𠺘口': 9.5,
 '极品基老伴：完结篇': 9.4,
 '拽妹黛薇儿要上大学了没': 9.4,
 '放牛班的春天': 9.3,
 '触不可及': 9.3,
 '鬼子来了': 9.3,
 '摩登时代': 9.3}

##  肯德基地址爬取

In [77]:
# 找到肯德基含有地址信息的网站，是一个自己输入城市然后网站返回列表的一个做法
# 输入北京之后，点击查询，发现网址并没有变，所以这是一个ajax请求

import requests

url = 'http://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

# 输入地址（城市）
city = input('please input the city name in Chinese: ')
param = {
    'cname': '',
    'pid': '',
    'keyword': city,
    'pageIndex': '1', # this indicate the number of pages you want
    'pageSize': '10'
}

response = requests.post(url = url, params = param, headers = headers)

page_text = response.text

print(page_text)

{"Table":[{"rowcount":88}],"Table1":[{"rownum":1,"storeName":"育慧里","addressDetail":"小营东路3号北京凯基伦购物中心一层西侧","pro":"24小时,Wi-Fi,店内参观,礼品卡","provinceName":"北京市","cityName":"北京市"},{"rownum":2,"storeName":"京通新城","addressDetail":"朝阳路杨闸环岛西北京通苑30号楼一层南侧","pro":"24小时,Wi-Fi,店内参观,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":3,"storeName":"黄寺大街","addressDetail":"黄寺大街15号北京城乡黄寺商厦","pro":"Wi-Fi,点唱机,店内参观,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":4,"storeName":"四季青桥","addressDetail":"西四环北路117号北京欧尚超市F1、B1","pro":"Wi-Fi,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":5,"storeName":"亦庄","addressDetail":"北京经济开发区西环北路18号F1＋F2","pro":"24小时,Wi-Fi,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":6,"storeName":"石园南大街","addressDetail":"通顺路石园西区南侧北京顺义西单商场石园分店一层、二层部分","pro":"24小时,Wi-Fi,店内参观,礼品卡,生日餐会","provinceName":"北京市","cityName":"北京市"},{"rownum":7,"storeName":"北京南站","addressDetail":"北京南站候车大厅B岛201号","pro":"Wi-Fi,礼品卡","provinceName":"北京市","cityName":"北京市"},{"rownum":8,

In [101]:
# 但这里page_text是str格式，想要转换成字典，要用json包
import json

real_page_text = json.loads(page_text)

street_name = []
for i in range(len(real_page_text['Table1'])):
    street_name.append(real_page_text['Table1'][i]['storeName'])

street_name

['育慧里',
 '京通新城',
 '黄寺大街',
 '四季青桥',
 '亦庄',
 '石园南大街',
 '北京南站',
 '北清路',
 '大红门新世纪肯德基餐厅',
 '巴沟']

## 药监总局的爬取

In [142]:
# 化妆品生产许可证的数据
# http://scxk.nmpa.gov.cn:81/xk/ 新网址
# 这个cell只负责把所有企业的名字提取出来

import requests

url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}


all_page = 10 # 一份工375页，但是不知道为啥当超过50页的时候会出现数据异常，所以我们只取前50页的

all_names = []

# 现在json_data['list']里面就是我需要的所有公司的信息了
for i in range(all_page):

    param = {
        'on': 'true',
        'page': i,
        'pageSize': '15',
        'productName': '',
        'conditionType': '1',
        'applyname': '',
        'applysn': '',
    }

    response = requests.post(url = url, params = param, headers= headers)
    json_data = response.json()
    
    for k in range(len(json_data['list'])):
        all_names.append(json_data['list'][k]['EPS_NAME'])
    
    # 加一个进度条
    # print(round(float(i/all_page), 4)*100, '% is done.')

all_names[:5]



['南京三盾药业有限公司',
 '青海环琼藏文化产品开发有限公司',
 '广东柏大化妆品有限公司',
 '南京佳原堂生物科技有限公司',
 '哈尔滨北星药业有限公司']

In [127]:
# 现在找到方法把每一家企业的详情给提取出来

import requests

url = 'http://scxk.nmpa.gov.cn:81/xk/#'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

param = {
    'on': 'true',
    'page': 1,
    'pageSize': '15',
    'productName': '',
    'conditionType': '1',
    'applyname': '',
    'applysn': '',
}

response = requests.get(url = url, headers = headers).text
# response

# 这里发现，如果你把这个东西转换成html然后与原网页对比，然后就会发现这里面不存在任何数据，但是原网页有，这就说明网页中的
# 数据是动态的，也就是说，想用药监总局的官网地址是请求不到数据的，需要用ajax才能提取到。

In [140]:
# 点开每一家公司的界面，看f12，发现所有数据都是动态的，param里面就是ID，所以直接摘取url，然后把ID放到param里，就OK了

import requests

url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

param = {
    'on': 'true',
    'page': '1',
    'pageSize': '15',
    'productName': '',
    'conditionType': '1',
    'applyname': '',
    'applysn': '',
}

response = requests.post(url = url, params = param, headers= headers)
json_data = response.json()

# 对每一家企业来说，url是（当发现网页是动态数据的时候得看ajax里面的网址了）
url_new = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsById'

# 输出前10个公司法人名字
legal_person = []
# 存储所有信息
all_info = []

for i in range(10): 

    id_company = {
        'id': json_data['list'][i]['ID']
    }

    # 我们现在取新的每个公司的信息
    response_company_page = requests.post(url = url_new, headers= headers, data = id_company) 
    # 这里注意一个细节，post()中的参数是data,而get()中是params!

    data_individual = response_company_page.json()
    
    legal_person.append(data_individual['legalPerson'])
    all_info.append(data_individual)
        
# 将所有信息保存
fp = open('./所有美妆公司信息.json', 'w', encoding = 'utf-8')
json.dump(all_info, fp = fp, ensure_ascii = False)

fp.close()

# 如果出现请求过多然后被拒绝的情况话，有可能可以试着加一个time.sleep(2)，看能不能解决

## 数据解析概述(即爬取整张页面之后，从中选取一小部分进行解析)

### 学习正则

In [105]:
# 数据解析分类:
# -正则
# vs4
# xpath（最重要，前两个看看就行了）

In [106]:
# 解析目标：网页中的文字，或者图片

In [113]:
# 先学一点正则
import re
key = 'javapythonc++'
print(re.findall('python', key)[0])

python


In [114]:
# 提取出hello world，夹在中间的！
key = '<html><h1>hello world<h1></html>'
re.findall('<h1>(.*)<h1>', key)[0]

'hello world'

In [116]:
# 提取170
key = '我喜欢身高170的女生'
re.findall('\d+', key)[0]

'170'

In [119]:
# 提取出网站名
key = 'http://www.baidu.com and https://boob.com'
re.findall('https?://', key)

['http://', 'https://']

In [124]:
# 提取出Hello，更改大小写
key = '<h1><hTmL>hello</HtML>'
re.findall('<[Hh][Tt][Mm][Ll]>(.*)</[Hh][Tt][Mm][Ll]>', key)

['hello']

In [129]:
# 提取gmail，这里其实就是提取g和.中间的东西
key = 'bobo@gmail.com'
re.findall('g.*?\.', key)

['gmail.']

In [130]:
# 匹配sas和saas
key = 'sas and assa and saas and saaas'
re.findall('sa{1,2}s', key)

['sas', 'saas']

### No.1，正则解析方式

In [134]:
# 目标：爬取图片数据https://www.qiushibaike.com/imgrank/
import requests
import re

url = 'https://pic.qiushibaike.com/system/pictures/12432/124320076/medium/HJAH0256Q9TK6IUM.jpg' # 随便找的一张图

# requests.get(url = url).text这里返回的是图片数据，如果用.text则会返回乱码，因为是二进制数据
# 所以这里用.content
image_data = requests.get(url = url).content # 返回二进制图片数据

# 存到本地
with open('./qiutu.jpg', 'wb') as fp:
    fp.write(image_data)
    fp.close()

In [162]:
# https://www.qiushibaike.com/imgrank/ 有35页，需要每一页的图片都下载下来
import requests
import re
import os # 这是为了直接创建一个文件夹！！！


url = 'https://www.qiushibaike.com/imgrank/'
# 对整张页面进行爬取


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}


page_text = requests.get(url = url, headers = headers).text # 获取网页源代码数据


# 现在使用聚焦爬虫找到所有的image然后解析
# 观察f12，发现所有图片都是类似这样
'''
<img src="//pic.qiushibaike.com/system/pictures/12431/124319738/medium/W7VD97D7DCI0PTOL.jpg" 
alt="糗事#124319738" class="illustration" width="100%" height="auto">
'''
# 即，都是img src开头的，所以直接找即可
# 观察一下，这里的层级问题，这个是div class等号后面的层级，比如当你选中div class = 'thumb'之后，就等于选中了thumb里面的全部数据
# 每个div下面，文字和图片都是分开储存在不同的div的！


# 现在开始提取
# ctrl+c 对div = thumb的，然后得到这个
'''
<div class="thumb">
<a href="/article/124319738" target="_blank">
<img src="//pic.qiushibaike.com/system/pictures/12431/124319738/medium/W7VD97D7DCI0PTOL.jpg" alt="糗事#124319738" class="illustration" width="100%" height="auto">
</a>
</div>
'''
# 现在提取src的值
# 对一个网页源代码来说，使用下面的ex，可以提取到所有src的值，有多少thumb就提取多少个！
ex = '<div class="thumb">.*?<img src="(.*?)" alt.*?</div>'


img_src_data = re.findall(ex, page_text, re.S) # 第三个参数是单行匹配，这里的一定不能使用re.M！
# 现在得到了所有url


# 先创建一个文件夹，用域放置图片
file_name = './糗图Libs'
if not os.path.exists(file_name): # 如果某个目录不存在，则创建
    os.mkdir(file_name)
    
    
# 对img_src_data遍历，然后全部get一遍，就可以下载所有图片了
for i in range(len(img_src_data)):
    # 这里img_src_data中的数据没有协议头，可能是http也可能是https，所以要先判断一下！
    real_url = 'http:' + img_src_data[i]
    
    image_data = requests.get(real_url, headers = headers).content # 千万不要用成text了！
    
    # 对每个图片取名，直接用split()切分！
    img_name = file_name + '/' + str(img_src_data[i].split('/')[-1]) # 改名，直接取根据/ split之后的最后一项！
    
    with open(img_name, 'wb') as fp:
        fp.write(image_data)
        print(img_src_data[i].split('/')[-1],'下载成功！')
        fp.close()

LN3YKS4W518Q150L.jpg 下载成功！
EZPSFRFNY792G0F1.jpg 下载成功！
5MJ9XMA6I1IPE890.jpg 下载成功！
VQA4T5IR6N84C6RH.jpg 下载成功！
HNQ8P34V20OL1P4Y 下载成功！
HJAH0256Q9TK6IUM.jpg 下载成功！
QQ6OQUYSEWCC8XU8.jpg 下载成功！
EAJ2VB7FKNRRE972.jpg 下载成功！
X42K2NCK0CCF1XGW.jpg 下载成功！
0EMDBRRKMYANUTII.jpg 下载成功！
MXAOR7QC4KNEW3FM.jpg 下载成功！
63Q6P7GQDD85DC2L.jpg 下载成功！
QXG8ZPIZQ9IY6QVZ.jpg 下载成功！
BLE9N8S5ML4DGFX6.jpg 下载成功！
XUMC6NV8WYO1JGS3.jpg 下载成功！
I9SFO8OVI3URQIGE.jpg 下载成功！
IZ38VODKZFY522X1.jpg 下载成功！
QFENIE2MLVCF16BI.jpg 下载成功！
FPVMQJJJ138GTN7M.jpg 下载成功！
SEGNC6WXW3ZRYLGK.jpg 下载成功！
QHP74EMKG1AIZP39.jpg 下载成功！
XTYQBVBEOSVBX613.jpg 下载成功！
JWPMA8JIIT38DSQK.jpg 下载成功！
II520ZELII50BY6M.jpg 下载成功！
KJ9N2GLJKQXB8DT4.jpg 下载成功！


In [168]:
# 现在我们对1-13页全部进行提取
# 点击页码，发现这个平台是直接给了新的网站，https://www.qiushibaike.com/imgrank/page/3/
# 所以可以直接更改url达到更改页码的目的

# 设置通用url模板！
# 使用占位符！
url = 'https://www.qiushibaike.com/imgrank/page/%d/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

file_name = './糗图Libs'
if not os.path.exists(file_name): # 如果某个目录不存在，则创建
    os.mkdir(file_name)

for pageNum in range(1, 3): # 从1到2页，不用下载太多
    new_url = format(url%pageNum) # 对应页码的url
    page_text = requests.get(url = new_url, headers = headers).text # 获取网页源代码数据
    
    ex = '<div class="thumb">.*?<img src="(.*?)" alt.*?</div>'

    img_src_data = re.findall(ex, page_text, re.S) # 第三个参数是单行匹配，这里的一定不能使用re.M！
    
    print('第', pageNum, '页正在下载！')
    for i in range(len(img_src_data)):
        real_url = 'http:' + img_src_data[i]
        image_data = requests.get(real_url, headers = headers).content
        img_name = file_name + '/' + str(img_src_data[i].split('/')[-1]) 

        with open(img_name, 'wb') as fp:
            fp.write(image_data)
            print(img_src_data[i].split('/')[-1],'下载成功！')
            fp.close()

第 1 页正在下载！
1JW09CHJAC0O6JAE.jpg 下载成功！
MXAOR7QC4KNEW3FM.jpg 下载成功！
XGP9IAGNHBKV7Y2U.jpg 下载成功！
7RI3V690QYOPJXPD.jpg 下载成功！
3W0D0QPUCX860C93.jpg 下载成功！
WF12UETMFI9RWAJE.jpg 下载成功！
QQ6OQUYSEWCC8XU8.jpg 下载成功！
ZO6X7F3870NW3HKN.jpg 下载成功！
AIYYZL98PL4P3BZY.jpg 下载成功！
4F3YU9UNO75G79E2.jpg 下载成功！
LN3YKS4W518Q150L.jpg 下载成功！
HJAH0256Q9TK6IUM.jpg 下载成功！
1ABO1M2MKCCCEMJY.jpg 下载成功！
P6M8FV857VD2Y7AB.jpg 下载成功！
N6B33U3Z7Y1J58UG.jpg 下载成功！
BXFX6PNFBPTWS0BB.jpg 下载成功！
ZQZC0FOCMI4ZPI6S.jpg 下载成功！
257IAXUD9I778KX0.jpg 下载成功！
DF5QOT5L7POLPFQ5.jpg 下载成功！
Q39IUFJUQ2ZZNYWE.jpg 下载成功！
W7VD97D7DCI0PTOL.jpg 下载成功！
PGWRRHHDYUV9EU1W.jpg 下载成功！
HN3O6XF11EFDCFJ9.jpg 下载成功！
KX0QRQDBFI2BLECT.jpg 下载成功！
H2UBQMEWM73D45XM.jpg 下载成功！
第 2 页正在下载！
6QV3X2AKN57F797S.jpg 下载成功！
L3NBZYY4PEVEORD3.jpg 下载成功！
OXM9R9S6TGKFX4ZC.jpg 下载成功！
8QZ76TU2X612MUT8.jpg 下载成功！
TWSCOKV35HHR4M06.jpg 下载成功！
XTYQBVBEOSVBX613.jpg 下载成功！
I9SFO8OVI3URQIGE.jpg 下载成功！
I8EZG6KODP3N9ZB6.jpg 下载成功！
UZ19LGYH96ZBO204.jpg 下载成功！
ZFY51R1MII5FYIWC.jpg 下载成功！
AGRCZUS4CBY3BCNI.jpg 下载成功！
5XKPVO

In [171]:
# 关于占位符
# %d	整数
# %f	浮点数
# %s	字符串
# %x	十六进制整数

format('https://www.qiushibaike.com/imgrank/page/%s/'%'aaa')

'https://www.qiushibaike.com/imgrank/page/aaa/'

### No.2，bs4解析方式

In [172]:
# bs4只能应用于python中
# 数据解析原理:
# 1. 标签定位；2.提取标签

# bs4原理：
# 1.实例化一个BeautifulSoup对象，并且将页面源码数据加载到该对象中
# 2.通过调用BeautifulSoup对象中相关的属性或者方法进行标签定位和数据提取

In [176]:
# 实例化一个BeautifulSoup对象，并且将源代码加载到这里面
from bs4 import BeautifulSoup

# 实例化
# 两种类型：1.将本地的html的数据加载到对象中，2.将互联网上获取的源码加载到BS中

# 先来测试本地的本路径的'狗-lizh.html'文件！
# 先打开本地文件，读取数据
fp = open('./搜狗-lizh.html', 'r', encoding = 'utf-8')

soup = BeautifulSoup(fp, 'lxml') # 第二个参数是固定的！这里soup就已经是网页源代码了！

fp.close()

'''
# 如果是互联网上得到的数据，一样处理
url = 'https://www.qiushibaike.com/imgrank/'
page_text = requests.get(url = url)
soup = BeautifulSoup(page_text.text, 'lxml')
'''

In [197]:
# 对本地保存的搜狗-lizhenning.html进行分析
from bs4 import BeautifulSoup

url = 'http://zhenning-li.com/'
source_text = requests.get(url = url, headers = headers).text

soup = BeautifulSoup(source_text, 'lxml') # 第二个参数是固定的！这里soup就已经是网页源代码了！

# soup.tagName返回的是，html中第一次出现的tagName的内容！
# 比如soup.div
# 还有一种一样的方法，是soup.find('div')
# find还可以属性定位。比如我的网站上，第一个div就是div，第二个里面就带有属性了，所以可以定位到第二个！

# soup.find_all()是返回符合要求的全部值，返回的是列表！

In [199]:
soup.find('div', class_ = 'main-header-bar-wrap') # 这里class_是带下划线的！
# 同理，这里的class_还可以是id_, attr_等，只要是网页上div之后出现的类别，都可以加！

<div class="main-header-bar-wrap">
<div class="main-header-bar">
<div class="ast-container">
<div class="ast-flex main-header-container">
<div class="site-branding">
<div class="ast-site-identity" itemscope="itemscope" itemtype="https://schema.org/Organization">
<span class="site-logo-img"><a aria-current="page" class="custom-logo-link" href="http://zhenning-li.com/" rel="home"><img alt="Zhenning's Place" class="custom-logo" height="50" sizes="(max-width: 50px) 100vw, 50px" src="https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/12/cropped-cropped-cropped-Vic-1-50x50.png" srcset="https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/12/cropped-cropped-cropped-Vic-1-50x50.png 50w, https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/12/cropped-cropped-cropped-Vic-1-300x300.png 300w, https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/1

In [204]:
len(soup.find_all('div'))

27

In [210]:
# select返回的是某一个选择器下面的内容！比如div后class是某个内容，然后select可以加这个内容，然后就extract出来了这个单独的部分！

soup.select('.site-branding') # 返回的是列表！而且这个点不能少！！！
# 但是这！！如果标签自带空格！像这样的 class = 'hfeed site'，则要在中间加'.'！soup.select('.hfeed.site')即可！！

[<div class="site-branding">
 <div class="ast-site-identity" itemscope="itemscope" itemtype="https://schema.org/Organization">
 <span class="site-logo-img"><a aria-current="page" class="custom-logo-link" href="http://zhenning-li.com/" rel="home"><img alt="Zhenning's Place" class="custom-logo" height="50" sizes="(max-width: 50px) 100vw, 50px" src="https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/12/cropped-cropped-cropped-Vic-1-50x50.png" srcset="https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/12/cropped-cropped-cropped-Vic-1-50x50.png 50w, https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/12/cropped-cropped-cropped-Vic-1-300x300.png 300w, https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/2020/12/cropped-cropped-cropped-Vic-1-150x150.png 150w, https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads

In [220]:
# select还能查找层级的关系！这个最重要！可以直接定位到比如div class下面的ul下面的li当中的某个网址等！
# 这里用 > 来表示层级关系！！
soup.select('.ast-main-header-bar-alignment > nav > div > ul > li > a') # 这个写的不对

# 如果想要跳跃层级，则直接用空格即可！soup.select('.ast-main-header-bar-alignment li')

[]

In [230]:
soup.select('.ast-main-header-bar-alignment li')

[<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-165" id="menu-item-165"><a class="menu-link" href="http://zhenning-li.com/about-me/">About Me</a></li>,
 <li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-181" id="menu-item-181"><a class="menu-link" href="http://zhenning-li.com/market-review/">Market Review</a></li>,
 <li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-180" id="menu-item-180"><a class="menu-link" href="http://zhenning-li.com/insights/">Insights</a></li>,
 <li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-183" id="menu-item-183"><a class="menu-link" href="http://zhenning-li.com/tutorials-of-anything-interesting/">Tutorials</a></li>,
 <li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-182" id="menu-item-182"><a class="menu-link" href="http://zhenning-li.com/disclaimer/">Disclaimer</a></li>,
 <li class="menu-item menu-item-type-custom m

In [232]:
soup.select('.hfeed.site')

[<div class="hfeed site" id="page">
 <a class="skip-link screen-reader-text" href="#content">Skip to content</a>
 <header class="site-header ast-primary-submenu-animation-slide-down header-main-layout-1 ast-primary-menu-enabled ast-logo-title-inline ast-hide-custom-menu-mobile ast-menu-toggle-icon ast-mobile-header-inline" id="masthead" itemid="#masthead" itemscope="itemscope" itemtype="https://schema.org/WPHeader">
 <div class="main-header-bar-wrap">
 <div class="main-header-bar">
 <div class="ast-container">
 <div class="ast-flex main-header-container">
 <div class="site-branding">
 <div class="ast-site-identity" itemscope="itemscope" itemtype="https://schema.org/Organization">
 <span class="site-logo-img"><a aria-current="page" class="custom-logo-link" href="http://zhenning-li.com/" rel="home"><img alt="Zhenning's Place" class="custom-logo" height="50" sizes="(max-width: 50px) 100vw, 50px" src="https://secureservercdn.net/160.153.137.99/h3e.ecb.myftpupload.com/wp-content/uploads/202

In [270]:
# 现在定位到某个标签之后，要提取这个标签中的文本内容
# 比如，我想提取zhenning-li.com的main-header的内容

header_name = []
page_text = soup.select('.ast-main-header-bar-alignment li')

for i in range(len(page_text)-1): # 这里注意，page_text中最后一个是无效的，要剔除！！
    
    ex = '<li class=.*?><a class.*?">(.*?)</a></li>'
    page_text_i = page_text[i]
    text_content = re.findall(ex, str(page_text_i), re.S)[0]
    
    header_name.append(text_content)
    
header_name

['About Me',
 'Market Review',
 'Insights',
 'Tutorials',
 'Disclaimer',
 'Linkedin',
 'Github']

In [285]:
# 这里有个简单一万倍的
header_name = []
page_text = soup.select('.ast-main-header-bar-alignment li')

for i in range(len(page_text)):
    header_name.append(page_text[i].text) # 不仅可以用.text，还可以用string
    
header_name

['About Me',
 'Market Review',
 'Insights',
 'Tutorials',
 'Disclaimer',
 'Linkedin',
 'Github',
 '\n\n\n\nSearch\n\n\n\n\nSearch for:\n\n\n\n \n']

In [287]:
header_name = []
page_text = soup.select('.ast-main-header-bar-alignment li')

for i in range(len(page_text)):
    header_name.append(page_text[i].string) # 不仅可以用.text，还可以用string。
                                            # 这里用string更好，因为这个html中最后一个没有内容
    
header_name

['About Me',
 'Market Review',
 'Insights',
 'Tutorials',
 'Disclaimer',
 'Linkedin',
 'Github',
 None]

In [288]:
header_name = []
page_text = soup.select('.ast-main-header-bar-alignment li')

for i in range(len(page_text)):
    header_name.append(page_text[i].get_text()) # 也可以用get_text()

header_name

['About Me',
 'Market Review',
 'Insights',
 'Tutorials',
 'Disclaimer',
 'Linkedin',
 'Github',
 '\n\n\n\nSearch\n\n\n\n\nSearch for:\n\n\n\n \n']

In [297]:
# text和get_text()可以get到全部的文本内容，包括不属于直系的标签内容！
# string只能获取该标签下面直系的文本！
# 一般来说用text的更普遍，因为很容易搞不懂哪些是直系的哪些不是。
page_text = soup.find('div', class_ = 'ast-main-header-bar-alignment')

# 此时，page_text.string就不会返回东西，因为直系的div下面没有任何东西！
page_text.text
# 会把所有下属的文本都返回了！

'About Me\nMarket Review\nInsights\nTutorials\nDisclaimer\nLinkedin\nGithub\n\n\n\n\nSearch\n\n\n\n\nSearch for:\n\n\n\n \n\n'

In [308]:
# 获取标签中的属性值
# soup.find_all('div')获取了所有div属性的源代码
# soup.find_all('div')[i]，获取了第i个源代码
# soup.find_all('div')[i]['class']直接第i个源代码的class类别中的属性名称

target_list = soup.find_all('div')
class_name = []
for i in range(len(target_list)):
    class_name.append(target_list[i]['class'])
class_name

[['hfeed', 'site'],
 ['main-header-bar-wrap'],
 ['main-header-bar'],
 ['ast-container'],
 ['ast-flex', 'main-header-container'],
 ['site-branding'],
 ['ast-site-identity'],
 ['ast-site-title-wrap'],
 ['ast-mobile-menu-buttons'],
 ['ast-button-wrap'],
 ['ast-main-header-bar-alignment'],
 ['main-header-bar-navigation'],
 ['main-navigation'],
 ['ast-search-menu-icon', 'slide-search'],
 ['ast-search-icon'],
 ['site-content'],
 ['ast-container'],
 ['content-area', 'primary'],
 ['entry-content', 'clear'],
 ['wp-block-coblocks-social',
  'wp-block-coblocks-social-profiles',
  'is-style-circular',
  'alignfull',
  'has-text-align-center',
  'has-colors'],
 ['widget-area', 'secondary'],
 ['sidebar-main'],
 ['ast-small-footer', 'footer-sml-layout-1'],
 ['ast-footer-overlay'],
 ['ast-container'],
 ['ast-small-footer-wrap'],
 ['ast-small-footer-section', 'ast-small-footer-section-1']]

In [316]:
soup.select('.ast-main-header-bar-alignment a')[0]['href']

'http://zhenning-li.com/about-me/'

### No.3，bs4案例！

In [371]:
# 爬取三国演义所有章节的名字以及内容！！https://www.shicimingju.com/book/sanguoyanyi.html
import requests
import re
import os
from bs4 import BeautifulSoup


url = 'https://www.shicimingju.com/book/sanguoyanyi.html'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}


page_text = requests.get(url = url, headers = headers)

page_text.encoding = 'utf-8' # 这里处理中文的关键！！！

page_text = page_text.text

soup = BeautifulSoup(page_text, 'lxml')

chapter_name = []
url_list_each_chapter = []

for i in range(len(soup.select('.book-mulu li'))):
    chapter_name.append(soup.select('.book-mulu li')[i].string)
    url_list_each_chapter.append('https://www.shicimingju.com'+soup.select('.book-mulu li')[i].a['href'])
    
chapter_name[:5]


['第一回·宴桃园豪杰三结义  斩黄巾英雄首立功',
 '第二回·张翼德怒鞭督邮    何国舅谋诛宦竖',
 '第三回·议温明董卓叱丁原  馈金珠李肃说吕布',
 '第四回·废汉帝陈留践位    谋董贼孟德献刀',
 '第五回·发矫诏诸镇应曹公  破关兵三英战吕布']

In [386]:
# 现在把每一章节的内容扒下来！
# 每一节的url已经在url_list_each_chapter里面了
import os

if not os.path.exists('./三国小说'):
    os.mkdir('./三国小说')

for i in range(len(url_list_each_chapter)-115): # 只让运行前10个看看
    fp = open('./三国小说/' + chapter_name[i] + '.md', 'w', encoding = 'utf-8')

    page_text = requests.get(url = url_list_each_chapter[i], headers = headers)
    page_text.encoding = 'utf-8'
    page_text = page_text.text

    soup = BeautifulSoup(page_text, 'lxml')

    target_text_total = soup.find('div', class_= 'chapter_content')
    # target_text_total = soup.select('.chapter_content') 也可以！
    
    page_content = target_text_total.text # 这一步是将内容给提取出来！！！不用烦心里面的/n什么的！！
    
    fp.write(chapter_name[i] + ':' + page_content)
    print(chapter_name[i], ' is finished.')
    
fp.close()

第一回·宴桃园豪杰三结义  斩黄巾英雄首立功  is finished.
第二回·张翼德怒鞭督邮    何国舅谋诛宦竖  is finished.
第三回·议温明董卓叱丁原  馈金珠李肃说吕布  is finished.
第四回·废汉帝陈留践位    谋董贼孟德献刀  is finished.
第五回·发矫诏诸镇应曹公  破关兵三英战吕布  is finished.


### No.4，xpath解析！

In [413]:
import requests
import os
from lxml import etree

# 我们先解析一下本地的数据

url = 'http://zhenning-li.com/'

page_text = requests.get(url = url).text

tree = etree.HTML(page_text)

# xpath只能根据层级来定位！
# /html前面加上/表示从根目录开始遍历！中间的/表示一个层级
tree.xpath('/html/body/div/header')

[<Element header at 0x2bbd21a34c0>]

In [414]:
tree.xpath('/html/body//nav//ul') # 中间的//表示多个层级！

[<Element ul at 0x2bbd23afd80>]

In [415]:
tree.xpath('//li') # 表示从任意位置直接定位li标签！！根据返回值的个数来判断是否正确

[<Element li at 0x2bbd243bac0>,
 <Element li at 0x2bbd243b340>,
 <Element li at 0x2bbd2436e40>,
 <Element li at 0x2bbd2436600>,
 <Element li at 0x2bbd2436780>,
 <Element li at 0x2bbd2436880>,
 <Element li at 0x2bbd2436e00>,
 <Element li at 0x2bbd2436d80>]

In [416]:
tree.xpath('//div[@class = "main-navigation"]') # 这样就实现了定位某个固定属性的标签！

[<Element div at 0x2bbd2436ac0>]

In [417]:
# 索引定位：一个标签下有很多个比如li这样的小标签时，定位到第n个！精确定位！
tree.xpath('//div[@class = "main-navigation"]//li[3]')  # 这就是获得了第三个标签！索引从1开始！

[<Element li at 0x2bbd2436e40>]

In [432]:
# 取文本，取属性
tree.xpath('//div[@class = "main-navigation"]//li[3]/a/text()')# /text()直接返回这个标签的直系文本！

['Insights']

In [419]:
tree.xpath('//div[@class = "main-navigation"]//li/a/text()') # 这样直接返回所有li下面的文本，以列表的形式！

['About Me',
 'Market Review',
 'Insights',
 'Tutorials',
 'Disclaimer',
 'Linkedin',
 'Github']

In [420]:
tree.xpath('//div[@class = "main-navigation"]//li//text()') # 获取标签中非直系的文本！

['About Me',
 'Market Review',
 'Insights',
 'Tutorials',
 'Disclaimer',
 'Linkedin',
 'Github',
 '\r\n\t\t\t\t\t\t',
 '\r\n\t\t\t',
 '\r\n\t\t\t\t',
 '\r\n\t\t\t\t\t',
 'Search',
 '\r\n\t\t\t\t',
 '\r\n\t\t\t',
 '\r\n\t\t\t',
 '\r\n\t\t\t',
 '\r\n\t\t\t\t',
 'Search for:',
 '\r\n\t\t\t\t',
 '\r\n\t\t\t',
 '\r\n\t\t\t',
 '\r\n\t\t',
 '\t\t',
 '\r\n\t\t\t\t\t']

In [429]:
# 获取li标签下面的a下面的href的值！
tree.xpath('//li[1]/a/@href')

['http://zhenning-li.com/about-me/']

In [431]:
# 所有网页的href值！
tree.xpath('//li/a/@href') # 取属性直接/@即可！

['http://zhenning-li.com/about-me/',
 'http://zhenning-li.com/market-review/',
 'http://zhenning-li.com/insights/',
 'http://zhenning-li.com/tutorials-of-anything-interesting/',
 'http://zhenning-li.com/disclaimer/',
 'https://www.linkedin.com/in/zhenning-li-522b83a4/',
 'https://github.com/handsomevictor']

### No.5，xpath案例！58同城！

In [487]:
# 58二手房案例！
# 目标，将所有房源名称解析出来即可！
import requests
from lxml import etree

url = 'https://bj.58.com/ershoufang/'

page_text = requests.get(url = url, headers = headers)
page_text.encoding = 'utf-8'
page_text = page_text.text

tree = etree.HTML(page_text)

In [490]:
# 这个case的难点在于，页面上的字并不是嵌入在本页面的Html中的，而是要根据里面的url到新的页面中去提取！

# 先把第一个页面的房源的url提取出来！

house_url_list = tree.xpath('//div[@class = "list"]//div[@class="property"]/a/@href')
house_url_list[:2]

['https://bj.58.com/ershoufang/46215853012365x.shtml?auction=200&hpType=60&entry=102&position=0&kwtype=filter&now_time=1621125615&typecode=200&lg=https%3A%2F%2Flegoclick.58.com%2Fjump%3Ftarget%3DszqBpB3draOWUvYfugF1pAqduh78uztYPWcQPH9dn1TQnWnvPg980v6YUykKuadBmH6-Pj9Yuad6PjuBsHwBm1NVrjDvmidbPyn3nW-Wrj9OPHTKn1DOP1NkP1cLn1cYnWnKTHDOrHndrj0dnjDYP1E3nWNKnWb1njTKnWb1njTKnikQnkDQTHDvnWDQnWNvnHEvnjmKnWnKwbPxwRPAgYQFNdwxNDnKnEDQTEDVnEDKnHnOn1EOn1TdPjmOrjT1P1cYrTDvTEDQTH93PHNkuW-BsyFhrAnVPAcOnad6nHczsHTLnHEOnhE1PjmznkDQn1b1Pjb1njNdnWD1rHnvPWEkTHD1rHnYrHnkPHNQn1TkPHEvPHmKTEDKTEDVTEDKTHbLwjEzPj9QsNNvnjTVPjTYniYOnju7sHEOnjK7Pbc1ENDdP9DYPB3QrHn8PWm8nWn3THTKnTDKnikQnk7exEDQnjT1n9DQnjTQrjn3THRBnyu6uhNzsymQrj0VPjKhmzYOrHnLsH9zn1PWmHmduhmznTDKnTDKTHTKnikQnHE1sjNOPWbKnE78IyQ_THcYrj7BujNYnhm1uWbOnHT&spread=filtersearch&from=from_esf_List_search&index=0',
 'https://bj.58.com/ershoufang/45597127559724x.shtml?auction=200&hpType=60&entry=102&position=1&kwtype=filter&now_time=1621125615&typecode=200&lg=https%3

In [491]:
url = house_url_list[0]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}

page_text = requests.get(url = url, headers = headers)
page_text.encoding = 'utf-8'
page_text = page_text.text

tree_2 = etree.HTML(page_text)
# 但是做到这里的时候！出现意外情况，那边识别了我们的爬虫。。。

In [496]:
# 可以这样
tree_2.xpath('//h1[@class="title"]//text()')

['香溪度 3室2厅2卫 品质小区 经典实用']

In [498]:
tree_2.xpath('//div[@class="props-main"]//h1//text()')

['香溪度 3室2厅2卫 品质小区 经典实用']

In [499]:
# 现在尝试一下能不能短时间内多请求几次

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}

house_name = []

for i in range(len(house_url_list)):
    url = house_url_list[i]

    page_text = requests.get(url = url, headers = headers)
    page_text.encoding = 'utf-8'
    page_text = page_text.text

    tree_2 = etree.HTML(page_text)
    
    house_name_single = tree_2.xpath('//div[@class="props-main"]//h1//text()')[0]
    house_name.append(house_name_single)
    
house_name
# 这里出现错误，意味着已经被发现了，这里发现是只能短时间内请求20次

IndexError: list index out of range

In [500]:
tree_2.xpath('//div[@class="props-main"]//h1//text()')
# 这里已经看不到任何东西了

[]

In [503]:
len(house_name)

20

In [502]:
len(house_url_list)

120

### No.6，xpath案例！下载4K图片！！

In [508]:
import requests
import os
from lxml import etree

url = 'https://pic.netbian.com/4kmeinv/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}


page_text = requests.get(url = url, headers = headers)
page_text.encoding = 'utf-8'
page_text = page_text.content

In [544]:
tree = etree.HTML(page_text)

url_title = 'https://pic.netbian.com'

if not os.path.exists('./netpian'):
    os.mkdir('./netpian')
        
file_path = './netpian/'

for i in range(len(tree.xpath('//div[@class="slist"]//li'))):
    
    # 这里注意！在xpath中如果要用循环，则一定要拼接！更好的办法可以直接全部提取出来之后直接放到list中！更快
    url_full = url_title + tree.xpath('//div[@class="slist"]//li['+ str((i+1)) + ']/a/img/@src')[0]

    image_data = requests.get(url = url_full, headers = headers).content

    # 给图片命名，直接选取网页中的名字
    tree = etree.HTML(page_text)
    image_name = tree.xpath('//div[@class="slist"]//li[' + str(i+1) + ']/a/b/text()')[0]

    with open(file_path + image_name + '.jpg', 'wb') as fp:
        fp.write(image_data)
        fp.close()

In [546]:
# 在做循环的时候，更推荐这样做！
tree = etree.HTML(page_text)

url_title = 'https://pic.netbian.com'

if not os.path.exists('./netpian'):
    os.mkdir('./netpian')
        
file_path = './netpian/'

for li in tree.xpath('//div[@class="slist"]//li'):
    image_data = li.xpath('./a/img/@src')[0]
    image_src = url_title + image_data
    
    image_name = li.xpath('./a/img/@alt')[0]
    image_content = requests.get(url = image_src, headers = headers).content # content这里就不需要转换成utf-8了
    
    with open(file_path + image_name + '.jpg', 'wb') as fp:
        fp.write(image_content)
        # print(image_name, ' success!')
        fp.close()

### No.7，xpath案例！全国城市名称爬取

In [548]:
# 爬取全国城市名称，这个难点在于，页面上的层级关系非常复杂，而且城市名称是按首字母排序的
import requests
from lxml import etree

url = 'https://www.aqistudy.cn/historydata/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}

page_text = requests.get(url = url, headers = headers)
page_text.encoding = 'utf-8'
page_text = page_text.text
tree = etree.HTML(page_text)

In [564]:
total_cities = tree.xpath('//body//div[@class="all"]//ul/div/li/a//text()')
total_cities[:10]

['阿坝州', '安康', '阿克苏地区', '阿里地区', '阿拉善盟', '阿勒泰地区', '安庆', '安顺', '鞍山', '克孜勒苏州']

In [565]:
hot_cities = tree.xpath('//body//div[@class="hot"]//ul/li/a//text()')
hot_cities

['北京', '上海', '广州', '深圳', '杭州', '天津', '成都', '南京', '西安', '武汉']

<p>现在出现了一个比较好玩的情况
<p>点开一个城市，发现f12调用不了，此时有三种解决办法：
<p>以火狐浏览器为例：</p> 
<p>方法一：鼠标点击网址栏，然后再按F12。目前不清楚这是个例还是通用的，因为我就遇到了一个这种网页。</p> 
<p>方法二：Ctrl+Shift+i。</p> 
<p>方法三：打开菜单-&gt;web开发者-&gt;web开发者工具，这个就是上面那个快捷键。</p> 
<hr> 
<p>以谷歌浏览器为例：</p> 
<p>方法一：依旧可以</p> 
<p>方法二：依旧可以</p> 
<p>方法三：自定义及控制-&gt;更多工具-&gt;开发者工具。</p> 
<hr> 
<p>作为一个爬虫选手，如果连审查页面元素的能力都没有，那也就不要干了嘛。</p>
<p>https://blog.csdn.net/weixin_46155246/article/details/108594993 也可以参考这个

In [ ]:
# 现在点开上海的网页，发现里面有历史数据，想要提取出来。但是发现似乎挺难的。。。

In [567]:
url = 'https://www.aqistudy.cn/historydata/monthdata.php?city=%E4%B8%8A%E6%B5%B7'
page_text = requests.get(url = url).text

In [587]:
tree = etree.HTML(page_text)
# tree.xpath('//div[@class="container"]//tr[1]//text()')
tree.xpath('//div[@class="container"]//tr[@height="38px"]/th/text()')

['月份', 'AQI', '范围', '质量等级', 'PM2.5', 'PM10', 'SO2', 'CO', 'NO2', 'O3']

## 自己写的一个爬取法国疫苗是否available的script！！是纯粹自己写着玩没啥技术含量

In [16]:
# The following codes only help you find if there are empty slots in 3 stations near Cergy, the three stations are
# goussainville, stade, and SDIS 60 (first one most preferred). 

import requests
import time
from datetime import date, timedelta

localtime = time.localtime(time.time())
localtime.tm_year

today = str(localtime.tm_year)+'-'+str(localtime.tm_mon)+'-'+str(localtime.tm_mday)
tomorrow = (date.today() + timedelta(days=1)).strftime("%Y-%m-%d")
next_week = (date.today() + timedelta(days=7)).strftime("%Y-%m-%d")

if __name__ == '__main__':
    check_today()
    check_tomorrow()
    check_next_week()

    import time
    count = 0
    b = 10
    print('after 10 seconds, this window will be closed.')
    while (count < b):
        ncount = b - count
        print(ncount, ' seconds left')
        time.sleep(1)
        count += 1

    print('bye bye!')
    time.sleep(1)

SDIS 60 is available today!
SDIS 60 is available tomorrow!
SDIS 60 is available 7 days later!
after 10 seconds, this window will be closed.
10  seconds left
9  seconds left
8  seconds left
7  seconds left
6  seconds left
5  seconds left
4  seconds left
3  seconds left
2  seconds left
1  seconds left
bye bye!


In [9]:
# functions to check if there are empty slots available today, tomorrow and in 7 days

def check_today():

    localtime = time.localtime(time.time())
    localtime.tm_year

    today = str(localtime.tm_year)+'-'+str(localtime.tm_mon)+'-'+str(localtime.tm_mday)
    
    if vaccination_goussainville(today) == True:
        print('goussainville is available today!')
    elif vaccination_stade(today) == True:
        print('stade is available today!')
    elif vaccination_SDIS_60(today) == True:
        print('SDIS 60 is available today!')
    else:
        print('No one is available today!')

def check_tomorrow():
    
    tomorrow = (date.today() + timedelta(days=1)).strftime("%Y-%m-%d")
    
    if vaccination_goussainville(today) == True:
        print('goussainville is available tomorrow!')
    elif vaccination_stade(today) == True:
        print('stade is available tomorrow!')
    elif vaccination_SDIS_60(today) == True:
        print('SDIS 60 is available tomorrow!')
    else:
        print('No one is available tomorrow!')
        
def check_next_week():
    
    next_week = (date.today() + timedelta(days=7)).strftime("%Y-%m-%d")
    
    if vaccination_goussainville(today) == True:
        print('goussainville is available 7 days later!')
    elif vaccination_stade(today) == True:
        print('stade is available 7 days later!')
    elif vaccination_SDIS_60(today) == True:
        print('SDIS 60 is available 7 days later!')
    else:
        print('No one is available 7 days later!')

In [23]:
import time
from datetime import date, timedelta

localtime = time.localtime(time.time())
localtime.tm_year

today = str(localtime.tm_year)+'-'+str(localtime.tm_mon)+'-'+str(localtime.tm_mday)
tomorrow = (date.today() + timedelta(days=1)).strftime("%Y-%m-%d")
next_week = (date.today() + timedelta(days=7)).strftime("%Y-%m-%d")

In [18]:
# City of Goussainville的站点
def vaccination_goussainville(today):
    import requests

    url = 'https://www.doctolib.fr/availabilities.json?start_date=2021-05-12&visit_motive_ids=2858018&agenda_ids=468303-468302-461143-468300-461141-461144-461147-468301-463304-461145-468304&insurance_sector=public&practice_ids=163985&destroy_temporary=true&limit=7'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }

    param = {
        'start_date': today,
        'visit_motive_ids': '2858018',
        'agenda_ids': '468303-468302-461143-468300-461141-461144-461147-468301-463304-461145-468304',
        'insurance_sector': 'public',
        'practice_ids': '163985',
        'destroy_temporary': 'true',
        'limit': '7'
    }


    response = requests.get(url = url, params = param, headers= headers)
    json_data = response.json()

    if json_data['total'] != 0:
        # print('You can apply the one in centre de vaccination ville de goussainville at ', today,'!!! Hurry up!!!')
        return True
    else:
        # print('Now it\'s only ', today, 'no available ones, please just wait more')
        return False

In [12]:
# 19-stade-de-france
def vaccination_stade(today):
    import requests

    url = 'https://www.doctolib.fr/availabilities.json?start_date=2021-05-12&visit_motive_ids=2735278&agenda_ids=447491-447376-447377-447378-447388-447389-447391-447375-447373-447381-447384-447385-447492-472647-472652-447380-447390-472653-472655-447383-447490-472658-469829-470547-470548-470549-469823-469824-469825-469827-469831-469836&insurance_sector=public&practice_ids=179894&destroy_temporary=true&limit=7'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }

    param = {
        'start_date': today,
        'visit_motive_ids': '2735278',
        'agenda_ids': '447491-447376-447377-447378-447388-447389-447391-447375-447373-447381-447384-447385-447492-472647-472652-447380-447390-472653-472655-447383-447490-472658-469829-470547-470548-470549-469823-469824-469825-469827-469831-469836',
        'insurance_sector': 'public',
        'practice_ids': '179894',
        'destroy_temporary': 'true',
        'limit': '7'
    }

    response = requests.get(url = url, params = param, headers= headers)
    json_data = response.json()


    if json_data['total'] != 0:
        #print('You can apply in stade!!! Hurry up!!!')
        return True
    else:
        #print('Just wait more until next week, let me check if it\'s available next week, please wait...')
        if stade_next_week()[0] == True:
            #print('Oh my god!! There are ', stade_next_week()[1], ' slots available at stade next week!')
        #else:
            #print('You can\'t apply even in next week, go to sleep man!!')
            return False

In [13]:
def stade_next_week():
    url = 'https://www.doctolib.fr/availabilities.json?start_date=2021-05-23&visit_motive_ids=2735278&agenda_ids=447491-447376-447377-447378-447388-447389-447391-447375-447373-447381-447384-447385-447492-472647-472652-447380-447390-472653-472655-447383-447490-472658-469829-470547-470548-470549-469823-469824-469825-469827-469831-469836&insurance_sector=public&practice_ids=179894&destroy_temporary=true&limit=7'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }

    param = {
        'start_date': '2021-05-23',
        'visit_motive_ids': '2735278',
        'agenda_ids': '447491-447376-447377-447378-447388-447389-447391-447375-447373-447381-447384-447385-447492-472647-472652-447380-447390-472653-472655-447383-447490-472658-469829-470547-470548-470549-469823-469824-469825-469827-469831-469836',
        'insurance_sector': 'public',
        'practice_ids': '179894',
        'destroy_temporary': 'true',
        'limit': '7'
    }

    response = requests.get(url = url, params = param, headers= headers)
    json_data = response.json()

    if json_data['total'] == 0:
        #print('You can\'t apply even in next week, go to sleep man!!')
        return False, 0
    else:
        #print('Oh my god!! There are ', json_data['total'], ' slots available next week!')
        return True, json_data['total']

In [14]:
def vaccination_SDIS_60(today):
    url = 'https://www.doctolib.fr/availabilities.json?start_date=2021-05-12&visit_motive_ids=2810290&agenda_ids=467244-467245-458892-458893-458894-467219&insurance_sector=public&practice_ids=183218&destroy_temporary=true&limit=7'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }

    param = {
        'start_date': today,
        'visit_motive_ids': '2810290',
        'agenda_ids': '467244-467245-458892-458893-458894-467219',
        'insurance_sector': 'public',
        'practice_ids': '183218',
        'destroy_temporary': 'true',
        'limit': '7'
    }

    response = requests.get(url = url, params = param, headers= headers)
    json_data = response.json()

    if json_data['total'] == 0:
        #print('You can\'t apply at SDIS 60 at ', today, ', go to sleep!!')
        return False
    else:
        #print('Oh my god!! Please go check SDIS 60!! I think there are some empty slots!!')
        return True

In [21]:
# 自己试试能不能爬法国这个网站
# City of Goussainville的站点

import requests

url = 'https://www.doctolib.fr/availabilities.json?start_date=2021-05-12&visit_motive_ids=2858018&agenda_ids=468303-468302-461143-468300-461141-461144-461147-468301-463304-461145-468304&insurance_sector=public&practice_ids=163985&destroy_temporary=true&limit=7'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}

param = {
    'start_date': today,
    'visit_motive_ids': '2858018',
    'agenda_ids': '468303-468302-461143-468300-461141-461144-461147-468301-463304-461145-468304',
    'insurance_sector': 'public',
    'practice_ids': '163985',
    'destroy_temporary': 'true',
    'limit': '7'
}


response = requests.get(url = url, params = param, headers= headers)
json_data = response.json()

if json_data['total'] != 0:
    print('You can apply the one in centre de vaccination ville de goussainville at ', today,'!!! Hurry up!!!')

else:
    print('Now it\'s only ', today, 'no available ones, please just wait more')

Now it's only  2021-6-2 no available ones, please just wait more


In [22]:
vaccination_stade(today)

True

## 验证码识别！

### 古诗文提取验证码！

In [594]:
# 验证码是一种反爬机制
# 登录操作可以用requests完成
url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx'
password = 'niubiniubi'
account = 'handsomevictor@foxmail.com'

In [598]:
# 三方平台，云打码，可以帮助识别验证码，文字类型，计算题等，原理仅仅是图像识别！
from yundama.Yundama import Yundama
yundama = Yundama(app_id="129953", app_key="z8nXa5O4+t0Z6zGonnVvyX3uMrKuQaaq")

In [615]:
yundama.get_balance()

{'RetCode': '0',
 'ErrMsg': 'succ',
 'RequestId': '',
 'RspData': {'cust_val': 990}}

In [ ]:
img = open('../data/image.png', 'rb')
yundama.get_code_result(img.read(), '验证码类型')

# 验证码类型查询：http://docs.fateadm.com/web/#/1?page_id=36

In [601]:
# 现在先把文件的验证码图片提取出来
import requests
from lxml import etree

# 登陆网站
url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

page_text = requests.get(url = url, headers = headers)
page_text.encoding = 'utf-8'
page_text = page_text.text

In [612]:
import os

tree = etree.HTML(page_text)

url_head = 'https://so.gushiwen.cn'
url_tail = tree.xpath('//div[@class="mainreg2"]/div[@class="mainreg2"][3]/img[@id="imgCode"]/@src')[0]

url_image = url_head + url_tail

# 下载image
image_content = requests.get(url = url_image, headers = headers).content

if not os.path.exists('./验证码'):
    os.mkdir('./验证码')

with open('./验证码/verification_image.jpg', 'wb') as fp:
    fp.write(image_content)
    fp.close

In [614]:
# 现在图片已经提取出来了
img = open('./验证码/verification_image.jpg', 'rb')
result = yundama.get_code_result(img.read(), '30400')

In [630]:
code_result = result['RspData'].split('"')[3]
code_result

'518u'

### 人人网模拟登录！(发现图片都被加密，所以改为古诗文登录)

In [676]:
# 开始模拟登录
# 爬取基于某些用户的个人信息
# 先点击登录之后查看f12然后发现是post请求
# 发现验证码是post请求中的一个属性，每一次请求都动态变化

# 第一步：对验证码进行识别
# 第二步：对post请求进行发送
# 第三步：对响应出来的数据进行存储

# 现在整体提取验证码（与上面一样）
def get_code_result():
    import requests
    from lxml import etree
    from yundama.Yundama import Yundama
    yundama = Yundama(app_id="129953", app_key="z8nXa5O4+t0Z6zGonnVvyX3uMrKuQaaq")

    # 登陆网站
    url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
    }

    page_text = requests.get(url = url, headers = headers)
    page_text.encoding = 'utf-8'
    page_text = page_text.text

    import os

    tree = etree.HTML(page_text)

    url_head = 'https://so.gushiwen.cn'
    url_tail = tree.xpath('//div[@class="mainreg2"]/div[@class="mainreg2"][3]/img[@id="imgCode"]/@src')[0]

    url_image = url_head + url_tail

    # 下载image
    image_content = requests.get(url = url_image, headers = headers).content

    if not os.path.exists('./验证码'):
        os.mkdir('./验证码')

    with open('./验证码/verification_image.jpg', 'wb') as fp:
        fp.write(image_content)
        fp.close

    # 现在图片已经提取出来了
    img = open('./验证码/verification_image.jpg', 'rb')
    result = yundama.get_code_result(img.read(), '30400')

    code_result = result['RspData'].split('"')[3]
    
    return code_result

In [693]:
# 进行登录

# 从f12找到login


import requests
from lxml import etree
from yundama.Yundama import Yundama
yundama = Yundama(app_id="129953", app_key="z8nXa5O4+t0Z6zGonnVvyX3uMrKuQaaq")

# 登陆网站
url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36'
}

page_text = requests.get(url = url, headers = headers)
page_text.encoding = 'utf-8'
page_text = page_text.text

import os

tree = etree.HTML(page_text)

url_head = 'https://so.gushiwen.cn'
url_tail = tree.xpath('//div[@class="mainreg2"]/div[@class="mainreg2"][3]/img[@id="imgCode"]/@src')[0]

url_image = url_head + url_tail

# 下载image
image_content = requests.get(url = url_image, headers = headers).content

if not os.path.exists('./验证码'):
    os.mkdir('./验证码')

with open('./验证码/verification_image.jpg', 'wb') as fp:
    fp.write(image_content)
    fp.close

# 现在图片已经提取出来了
img = open('./验证码/verification_image.jpg', 'rb')
result = yundama.get_code_result(img.read(), '30400')

code_result = result['RspData'].split('"')[3]



url_login = 'https://so.gushiwen.cn/user/login.aspx?from=http%3a%2f%2fso.gushiwen.cn%2fuser%2fcollect.aspx'

data = {
    '__VIEWSTATE': 'EEeM2jY+85+50CxO1Mvg6cYe4nXXAOGlgzVz4f9YaeqMdI44DxZPxIc5kp7/YIgAucbyOWtaHYfVu7x1H/YumnaLFULs8lIRC7B2bBxqXsq91iQ7453qQTIzgXc=',
    '__VIEWSTATEGENERATOR': 'C93BE1AE',
    'from': 'http://so.gushiwen.cn/user/collect.aspx',
    'email': 'handsomevictor@foxmail.com',
    'pwd': 'niubiniubi',
    'code': code_result,
    'denglu': '登录'
}

data = {
    '__VIEWSTATE': 'EEeM2jY%2B85%2B50CxO1Mvg6cYe4nXXAOGlgzVz4f9YaeqMdI44DxZPxIc5kp7%2FYIgAucbyOWtaHYfVu7x1H%2FYumnaLFULs8lIRC7B2bBxqXsq91iQ7453qQTIzgXc%3D',
    '__VIEWSTATEGENERATOR': 'C93BE1AE',
    'from': 'http%3A%2F%2Fso.gushiwen.cn%2Fuser%2Fcollect.aspx',
    'email': 'handsomevictor%40foxmail.com',
    'pwd': 'niubiniubi',
    'code': code_result.upper(),
    'denglu': '%E7%99%BB%E5%BD%95'
    
}


# 登录成功之后的页面！
login_page_text = requests.post(url = url_login, data = data, headers = headers)
print(login_page_text.status_code) # 响应码如果是200则代表响应成功！


login_page_text = requests.post(url = url_login, data = data, headers = headers).text

200


In [694]:
# 登陆成功后主页面上有一个图片，提取出来

from lxml import etree

tree = etree.HTML(login_page_text)
tree.xpath('//div[@class="left"]//text()')

[]

正式宣布，挑战失败

## 模拟登录cookies操作！

In [22]:
import json
from lxml import etree

f=open(".\吉刻 Right Now.html","rb")
content=f.read().decode('utf-8')
tree=etree.HTML(content)
tree

<Element html at 0x2a52829c180>

In [18]:
url = 'C:\\Users\\handsomevictor\\OneDrive - CentraleSupelec\\Bureau\\Python\\python自己的教程\\爬虫\\吉刻 Right Now.html'

from bs4 import BeautifulSoup

page_text = etree.parse('.\吉刻 Right Now.html', etree.HTMLParser())

result = etree.tostring(page_text)

In [35]:
from lxml import etree

result = tree.xpath('//*[@id="gwc-table"]/tbody/tr//div/h3/text()')
price = tree.xpath('//*[@id="gwc-table"]/tbody/tr/td[2]/text()')
amount = tree.xpath('//*[@id="gwc-table"]/tbody/tr/td[3]/text()')
total_amount  = tree.xpath('//*[@id="totalMoney"]/text()')

In [56]:
import pandas as pd
import numpy as np

c = zip(result, price, amount)

df_jike = pd.DataFrame(c)

true_amount = []
for i in range(len(amount)):
    a = amount[i][:1]
    true_amount.append(a)

true_price = []
for i in range(len(price)):
    a = price[i][1:]
    true_price.append(a)

df_jike['true_amount'] = true_amount
df_jike['true_price'] = true_price

true_amount = np.array(true_amount)
true_price = np.array(true_price)

true_amount = true_amount.astype(np.float64)
true_price = true_price.astype(np.float64)

total_amount_for_each = true_amount * true_price
df_jike['total_amount_for_each'] = total_amount_for_each

df_jike = df_jike.drop([1, 2], axis = 1)
df_jike

,0,true_amount,true_price,total_amount_for_each
0,自然之源 红花椒/50g Sichuan Wild Pepper Red Whole50g,1,1.89,1.89
1,金凤凰贵族香米 1K Riz parfume PHO/OR1kg*10,1,2.69,2.69
2,小浣熊 销魂香辣蟹味干脆面/40g Crispy Instant NoodlesSpicy ...,6,0.79,4.74
3,康雅酷 冰粉粉 鲜橙味/40g Konjac Powder-Orange Flavour40g,3,0.69,2.07
4,康雅酷 冰粉粉 草莓味/40g Konjac Powder-Strawberry Flavo...,3,0.69,2.07
5,康师傅 3+2 香草口味 夹心卷 \n/55g Sandwich Roll Vanilla ...,3,1.69,5.07
6,午餐肉/梅林 Pate Porc Maling NL340g*12,1,4.19,4.19
7,旺旺仙贝/56g Cracker de riz WW56g,3,1.49,4.47
8,小浣熊 辣味帅锅味干脆面/40g Crispy Instant NoodlesSpicy F...,6,0.79,4.74
9,上好佳 鲜虾条/40g Oishi Prawn Crackers40g,2,1.49,2.98


In [57]:
df_jike['total_amount_for_each'].sum()

59.36

In [100]:
def target():
    pts = 10000
    normal_result = 5*np.random.normal(0, 1, size=pts)
    zero_list = np.zeros((pts,))

    residuals = normal_result - zero_list
    # create y = 10x+20
    x_value = np.arange(0,pts)
    y_value = 10*x_value + 20 + residuals

    # now we calculate OLS
    import statsmodels.api as sm
    x_model = sm.add_constant(x_value)
    y_model = y_value

    result = sm.OLS(y_model, x_model)
    result = result.fit()
    params_result = result.params

    nominal_residuals = x_value*params_result[1]+ params_result[0]-y_model
    nominal_residuals_shift = nominal_residuals[1:]
    nominal_residuals_shift = np.append(nominal_residuals_shift, 0)

    #epsilon t * t+1
    epsilon_result = nominal_residuals_shift * nominal_residuals

    return epsilon_result.sum()/pts

true_result = []
pts2 = 1000
for i in range(pts2):
    true_result.append(target())

sum(true_result)/pts2

0.001613110174653851

In [80]:
len(nominal_residuals_shift)

999